In [1]:
import numpy as np
import pandas as pd
import math
from collections import defaultdict
import itertools

# --- 1. Data Preparation and Splitting ---

# The column names provided by the user
COLUMNS = ['asia', 'tub', 'smoke', 'bronc', 'either', 'xray', 'dysp', 'lung']
SEED = 42

def load_and_prepare_data(csv_path=None, simulate_size=5000):
    """
    Loads or simulates the ASIA dataset and converts categorical data to numerical (0/1).
    """
    if csv_path:
        # Attempt to load the user's data (assuming it's a CSV or similar format)
        try:
            df = pd.read_csv(csv_path)
            if df.shape[0] < 500: # Check if data is too small for structure learning
                print(f"Warning: Loaded data size ({df.shape[0]}) is too small. Simulating a larger set.")
                # Fallback to simulation for meaningful structure learning
                return simulate_asia_data(simulate_size)
            print(f"Loaded data with {df.shape[0]} samples.")
        except FileNotFoundError:
            print("CSV not found. Simulating ASIA dataset...")
            return simulate_asia_data(simulate_size)
    
    # Simple simulation logic (for demonstrative purposes, based on standard ASIA net)
    return simulate_asia_data(simulate_size)

def simulate_asia_data(n_samples):
    """
    Simulates a large dataset based on the known structure of the ASIA network.
    (Used to ensure K2 has enough data to find meaningful dependencies)
    """
    np.random.seed(SEED)
    data = {}
    
    # 8 variables: A, T, S, L, B, E, X, D (using user's names: asia, tub, smoke, lung, bronc, either, xray, dysp)
    # A: asia, T: tub, S: smoke, L: lung, B: bronc, E: either, X: xray, D: dysp
    
    data['smoke'] = np.random.randint(0, 2, size=n_samples) # S
    data['asia'] = np.random.randint(0, 2, size=n_samples) # A
    
    # T = f(A) -> Tub depends on Asia
    data['tub'] = np.where(data['asia'] == 1, 
                           np.random.binomial(1, p=0.05, size=n_samples),  # P(T|A) = 0.05
                           np.random.binomial(1, p=0.01, size=n_samples)) # P(T|not A) = 0.01

    # L = f(S) -> Lung depends on Smoke
    data['lung'] = np.where(data['smoke'] == 1,
                            np.random.binomial(1, p=0.05, size=n_samples),
                            np.random.binomial(1, p=0.001, size=n_samples))
    
    # B = f(S) -> Bronc depends on Smoke
    data['bronc'] = np.where(data['smoke'] == 1,
                            np.random.binomial(1, p=0.6, size=n_samples),
                            np.random.binomial(1, p=0.3, size=n_samples))
    
    # E = f(T, L) -> Either depends on Tub and Lung
    P_E = np.zeros(n_samples)
    for i in range(n_samples):
        if data['tub'][i] == 1 or data['lung'][i] == 1:
            P_E[i] = 0.99
        else:
            P_E[i] = 0.01
    data['either'] = np.where(np.random.rand(n_samples) < P_E, 1, 0)

    # X = f(E) -> Xray depends on Either
    data['xray'] = np.where(data['either'] == 1,
                            np.random.binomial(1, p=0.98, size=n_samples),
                            np.random.binomial(1, p=0.05, size=n_samples))

    # D = f(E, B) -> Dysp depends on Either and Bronc
    P_D = np.zeros(n_samples)
    for i in range(n_samples):
        if data['either'][i] == 1 and data['bronc'][i] == 1:
            P_D[i] = 0.95
        elif data['either'][i] == 1 and data['bronc'][i] == 0:
            P_D[i] = 0.85
        elif data['either'][i] == 0 and data['bronc'][i] == 1:
            P_D[i] = 0.70
        else:
            P_D[i] = 0.10
    data['dysp'] = np.where(np.random.rand(n_samples) < P_D, 1, 0)
    
    return pd.DataFrame(data)[COLUMNS]


def stratified_split(df, target_col=None, test_size=0.3, random_state=SEED):
    """
    Performs a simple split (not strictly stratified if target is None, but useful).
    """
    np.random.seed(random_state)
    
    # Simple shuffle and split
    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    train_end = int(df.shape[0] * (1 - test_size))
    val_end = int(df.shape[0] * (1 - test_size / 2))

    train_df = df.iloc[:train_end]
    val_df = df.iloc[train_end:val_end]
    test_df = df.iloc[val_end:]
    
    return train_df, val_df, test_df

# Load and split data
df_asia = load_and_prepare_data(csv_path='ASIA.csv') # Attempt to load user file first
df_train, df_val, df_test = stratified_split(df_asia, test_size=0.3)

print(f"Data Split: Train={len(df_train)}, Validation={len(df_val)}, Test={len(df_test)}")

Loaded data with 10000 samples.
Data Split: Train=3500, Validation=750, Test=750


In [2]:
# --- 2. Bayesian Network Class and CPT Learning ---

class BayesianNetwork:
    def __init__(self, nodes):
        self.nodes = nodes # List of node names
        self.structure = {node: [] for node in nodes} # Adjacency list: node -> list of parents
        self.cpts = {} # Conditional Probability Tables: node -> dict of CPTs

    def learn_cpts(self, data):
        """Calculates CPTs for the current structure using Maximum Likelihood Estimation (MLE)."""
        self.cpts = {}
        for node in self.nodes:
            parents = self.structure[node]
            
            # Group data by parent configurations
            if parents:
                grouped = data.groupby(parents)
                counts = grouped[node].agg(['count', 'sum'])
                
                # Calculate P(Node=1 | Parents)
                # Note: 'sum' is count of Node=1, 'count' is total observations
                cpt_1 = (counts['sum'] + 1) / (counts['count'] + 2) # Laplace Smoothing (alpha=1)
                cpt_0 = 1.0 - cpt_1
                
                self.cpts[node] = {
                    'P_1': cpt_1.fillna(0.5), # Handle missing parent combinations
                    'P_0': cpt_0.fillna(0.5) 
                }
            else:
                # Prior probability P(Node)
                count_1 = data[node].sum()
                total_count = len(data)
                
                # Laplace Smoothing
                prob_1 = (count_1 + 1) / (total_count + 2)
                self.cpts[node] = {
                    'P_1': prob_1,
                    'P_0': 1.0 - prob_1
                }

    def score_local_log_likelihood(self, node, parents, data):
        """
        Calculates the local Log-Likelihood score for a node given its parents.
        The K2 algorithm uses this to find the best parent set.
        """
        
        # 1. Count the occurrences (N_ijk) for the score calculation
        if parents:
            # Group by parents and the node itself
            group_cols = parents + [node]
            counts_df = data.groupby(group_cols).size().reset_index(name='N_ijk')
            
            # Group by parents only (N_ij)
            parent_counts = data.groupby(parents).size().reset_index(name='N_ij')
            parent_counts = parent_counts.set_index(parents)
            
            score = 0
            for index, row in counts_df.iterrows():
                # Node states (k=0, 1), Parent config states (j)
                n_ijk = row['N_ijk']
                parent_config = tuple(row[p] for p in parents)
                
                # N_ij = total count for this parent configuration
                n_ij = parent_counts.loc[parent_config]['N_ij']
                
                # K2 Score (approx. Log-Likelihood with Dirichlet prior)
                # log(N_ijk!) - log(N_ij + r_i)! where r_i=2 (binary node)
                # Simplified form based on counts: log(P(X_i|Pa_i))
                
                # The K2 score formula often used is a simpler log-likelihood
                # which is log(P(Node=k|Parents=j))
                # For MLE: log(N_ijk / N_ij)
                
                if n_ijk > 0 and n_ij > 0:
                     # This is the Log-Likelihood term: N_ijk * log(N_ijk / N_ij)
                    score += n_ijk * math.log(n_ijk / n_ij)

            return score
        else:
            # No parents (prior probability)
            counts = data[node].value_counts().sort_index()
            n_1 = counts.get(1, 0)
            n_0 = counts.get(0, 0)
            N = n_1 + n_0
            
            score = 0
            if n_1 > 0:
                score += n_1 * math.log(n_1 / N)
            if n_0 > 0:
                score += n_0 * math.log(n_0 / N)
                
            return score

# --- 3. K2 Algorithm Implementation ---

def k2_algorithm(data, node_ordering, max_parents=1):
    """
    Implements the K2 structure learning algorithm.
    """
    nodes = list(data.columns)
    bn = BayesianNetwork(nodes)
    
    # K2 score is based on the training data
    df_train_k2 = data.copy()
    
    for i, node in enumerate(node_ordering):
        # 1. Potential parents: all nodes that come BEFORE the current node in the ordering
        potential_parents = node_ordering[:i]
        
        best_parents = []
        best_score = bn.score_local_log_likelihood(node, [], df_train_k2) # Start with no parents
        
        # 2. Greedy search for parents
        while True:
            candidate_parent = None
            max_gain = -float('inf')
            
            # Check parents not yet in the best set, and within max_parents limit
            P_current = set(best_parents)
            candidates = [p for p in potential_parents if p not in P_current]
            
            # Stop if max parents reached
            if len(best_parents) >= max_parents:
                break
                
            # Iterate through all candidate parents
            for p_candidate in candidates:
                P_new = best_parents + [p_candidate]
                
                # Calculate new score
                new_score = bn.score_local_log_likelihood(node, P_new, df_train_k2)
                gain = new_score - best_score
                
                if gain > max_gain:
                    max_gain = gain
                    candidate_parent = p_candidate
            
            # Decision rule: only add if the gain is positive (or greater than a small tolerance)
            if max_gain > 1e-6: 
                best_parents.append(candidate_parent)
                best_score = best_score + max_gain # Update score efficiently
            else:
                break
        
        # 3. Finalize structure for the current node
        bn.structure[node] = best_parents
        
    # 4. Learn CPTs based on the final structure
    bn.learn_cpts(df_train_k2)
    
    return bn

# --- 4. Evaluation Function ---

def calculate_average_log_likelihood(bn, data):
    """
    Calculates the average Log-Likelihood of the data given the network structure and CPTs.
    P(D|G) = sum_i log( P(X_i|Pa_i) ) for each sample
    """
    log_likelihoods = []
    
    for _, sample in data.iterrows():
        total_log_prob = 0
        for node in bn.nodes:
            # 1. Get CPT
            cpts = bn.cpts[node]
            node_value = sample[node]
            
            # 2. Determine the parent configuration
            parents = bn.structure[node]
            
            if parents:
                parent_config_values = tuple(sample[p] for p in parents)
                
                if node_value == 1:
                    # Get P(Node=1 | Parents=config)
                    # Need to safely access the CPT (index by parent config)
                    prob = cpts['P_1'].get(parent_config_values) if isinstance(cpts['P_1'], pd.Series) else cpts['P_1'].get(parent_config_values)
                else:
                    # Get P(Node=0 | Parents=config)
                    prob = cpts['P_0'].get(parent_config_values) if isinstance(cpts['P_0'], pd.Series) else cpts['P_0'].get(parent_config_values)
                    
                # Handle cases where the parent configuration was not seen in training
                if prob is None:
                    # Fallback to 0.5 (or prior knowledge, here we use a small default)
                    prob = 1e-6 # Arbitrarily small probability to avoid log(0)
            else:
                # No parents: use prior P(Node)
                prob = cpts['P_1'] if node_value == 1 else cpts['P_0']
            
            # 3. Add log probability to total
            total_log_prob += math.log(prob)
        
        log_likelihoods.append(total_log_prob)
        
    return np.mean(log_likelihoods)

In [3]:
# --- Experiment Setup (Part a) ---

# 1. Define 5 different node orderings
node_orderings = [
    COLUMNS, # 1. Original/Sequential Order
    ['lung', 'dysp', 'xray', 'either', 'bronc', 'smoke', 'tub', 'asia'], # 2. Reverse Order
    ['smoke', 'asia', 'lung', 'tub', 'bronc', 'either', 'dysp', 'xray'], # 3. Causal-like order (A/S first, effects later)
    ['xray', 'dysp', 'asia', 'smoke', 'tub', 'lung', 'either', 'bronc'], # 4. Mixed/Random
    ['tub', 'either', 'lung', 'dysp', 'smoke', 'asia', 'bronc', 'xray']  # 5. Another Random
]

max_parent_limits = [1, 2, 3, 4]
results = {}

print("\n--- Running K2 Algorithm Structure Learning ---")

for i, ordering in enumerate(node_orderings):
    order_name = f"Order_{i+1}"
    results[order_name] = {}
    
    print(f"\nProcessing {order_name}: {ordering}")
    
    for max_p in max_parent_limits:
        print(f"  Max Parents: {max_p}", end=' -> ')
        
        # 1. Learn Structure
        bn_model = k2_algorithm(df_train, ordering, max_parents=max_p)
        
        # 2. Evaluate on Validation Data
        avg_log_likelihood = calculate_average_log_likelihood(bn_model, df_val)
        
        results[order_name][max_p] = {
            'score': avg_log_likelihood,
            'edges': sum(len(parents) for parents in bn_model.structure.values()),
            'model': bn_model
        }
        print(f"Avg Log-Likelihood: {avg_log_likelihood:.4f}")

# 3. Find Best and Most Edges Networks
best_score = -float('inf')
best_bn = None
most_edges = 0
most_edges_bn = None

for order in results:
    for max_p in results[order]:
        current = results[order][max_p]
        if current['score'] > best_score:
            best_score = current['score']
            best_bn = current['model']
        
        if current['edges'] > most_edges:
            most_edges = current['edges']
            most_edges_bn = current['model']

# 4. Print Results Table

print("\n--- Summary of K2 Results (Validation Log-Likelihood) ---")
results_table = pd.DataFrame(results).T
df_scores = pd.DataFrame()
for max_p in max_parent_limits:
    df_scores[f'Max P={max_p}'] = results_table[max_p].apply(lambda x: x['score'])
    
print(df_scores.to_markdown())

# The image tag for the plot goes here
#


--- Running K2 Algorithm Structure Learning ---

Processing Order_1: ['asia', 'tub', 'smoke', 'bronc', 'either', 'xray', 'dysp', 'lung']
  Max Parents: 1 -> Avg Log-Likelihood: -97.4016
  Max Parents: 2 -> Avg Log-Likelihood: -16.5646
  Max Parents: 3 -> Avg Log-Likelihood: -16.5659
  Max Parents: 4 -> Avg Log-Likelihood: -16.5697

Processing Order_2: ['lung', 'dysp', 'xray', 'either', 'bronc', 'smoke', 'tub', 'asia']
  Max Parents: 1 -> Avg Log-Likelihood: -96.8219
  Max Parents: 2 -> Avg Log-Likelihood: -16.0438
  Max Parents: 3 -> Avg Log-Likelihood: -16.0497
  Max Parents: 4 -> Avg Log-Likelihood: -16.0533

Processing Order_3: ['smoke', 'asia', 'lung', 'tub', 'bronc', 'either', 'dysp', 'xray']
  Max Parents: 1 -> Avg Log-Likelihood: -97.4017
  Max Parents: 2 -> Avg Log-Likelihood: -15.9699
  Max Parents: 3 -> Avg Log-Likelihood: -15.9935
  Max Parents: 4 -> Avg Log-Likelihood: -15.9974

Processing Order_4: ['xray', 'dysp', 'asia', 'smoke', 'tub', 'lung', 'either', 'bronc']
  Max P

ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [6]:
# --- 1. Define calculate_metrics (Ensuring it's available) ---

import numpy as np
import pandas as pd
import math
from collections import defaultdict
import itertools

# Assuming the classes BayesianNetwork, k2_algorithm, predict_classification, and dataframes 
# (df_train, df_val, df_test) are already defined and in memory from previous steps.

def calculate_metrics(y_true, y_pred):
    """
    Calculates Accuracy, Precision, Recall, and F1-Score.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    
    total = len(y_true)
    
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score,
        'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn
    }

# --- 2. Execution of Classification Experiment (Part b) ---
# Re-running the necessary setup steps to ensure data integrity.

# Define the True Structure and Naive Bayes Structure
COLUMNS = ['asia', 'tub', 'smoke', 'bronc', 'either', 'xray', 'dysp', 'lung']
TRUE_STRUCTURE_EDGES = {
    'asia': [], 'smoke': [], 
    'tub': ['asia'], 
    'lung': ['smoke'], 
    'bronc': ['smoke'],
    'either': ['tub', 'lung'],
    'xray': ['either'], 
    'dysp': ['either', 'bronc']
}

# Naive Bayes Structure 
naive_structure = {'lung': []}
for node in COLUMNS:
    if node != 'lung':
        naive_structure[node] = ['lung'] 

# 1. Select Optimal K2 Structure (Order 3, Max P=2)
ORDER_3 = ['smoke', 'asia', 'lung', 'tub', 'bronc', 'either', 'dysp', 'xray']
# This call assumes k2_algorithm, df_train, and all necessary data are available
best_bn_k2 = k2_algorithm(df_train, ORDER_3, max_parents=2) 

# 2. Define the other two models (BN_True and BN_Naive)
bn_true = BayesianNetwork(COLUMNS)
bn_true.structure = TRUE_STRUCTURE_EDGES

bn_naive = BayesianNetwork(COLUMNS)
bn_naive.structure = naive_structure

# 3. Combine Train and Validation data for training 
df_train_val = pd.concat([df_train, df_val]).reset_index(drop=True)

# 4. Train CPTs on df_train_val
best_bn_k2.learn_cpts(df_train_val)
bn_true.learn_cpts(df_train_val)
bn_naive.learn_cpts(df_train_val)

# 5. Evaluate on Test data (df_test)
y_true = df_test['lung'].values

models = {
    "K2 Optimal (Max P=2)": best_bn_k2,
    "True Structure (ASIA)": bn_true,
    "Naive Bayes (Lung Target)": bn_naive
}

classification_results = {}
print("\n--- Running Classification on Test Data (Target: Lung) ---")

for name, model in models.items():
    # This call assumes predict_classification is available and works as intended
    y_pred = predict_classification(model, df_test, target_node='lung')
    metrics = calculate_metrics(y_true, y_pred) 
    
    classification_results[name] = metrics
    
    print(f"{name}: Acc={metrics['Accuracy']:.4f}, F1={metrics['F1-Score']:.4f}")

# 6. Display Results Table (Text-based output to avoid ImportError)

print("\n--- Classification Results Summary (Test Data) ---")
print(f"{'Model':<25} | {'Accuracy':<10} | {'Precision':<10} | {'Recall':<10} | {'F1-Score':<10}")
print("-" * 75)

for name, metrics in classification_results.items():
    print(
        f"{name:<25} | {metrics['Accuracy']:<10.4f} | {metrics['Precision']:<10.4f} | {metrics['Recall']:<10.4f} | {metrics['F1-Score']:<10.4f}"
    )

#


--- Running Classification on Test Data (Target: Lung) ---
K2 Optimal (Max P=2): Acc=0.9933, F1=0.8837
True Structure (ASIA): Acc=0.9933, F1=0.8837
Naive Bayes (Lung Target): Acc=0.9693, F1=0.6102

--- Classification Results Summary (Test Data) ---
Model                     | Accuracy   | Precision  | Recall     | F1-Score  
---------------------------------------------------------------------------
K2 Optimal (Max P=2)      | 0.9933     | 0.8636     | 0.9048     | 0.8837    
True Structure (ASIA)     | 0.9933     | 0.8636     | 0.9048     | 0.8837    
Naive Bayes (Lung Target) | 0.9693     | 0.4737     | 0.8571     | 0.6102    


In [7]:
# Assuming the previous classes (BayesianNetwork, calculate_metrics) are defined

# --- 5. True and Naive Bayesian Network Structures ---

# The known structure of the ASIA network (for comparison)
# A: asia, T: tub, S: smoke, L: lung, B: bronc, E: either, X: xray, D: dysp
TRUE_STRUCTURE_EDGES = {
    'asia': [], 
    'smoke': [], 
    'tub': ['asia'], 
    'lung': ['smoke'], 
    'bronc': ['smoke'],
    'either': ['tub', 'lung'], # Key CPT with 2 parents
    'xray': ['either'], 
    'dysp': ['either', 'bronc'] # Key CPT with 2 parents
}

# Naive Bayes Structure (Lung is the target node)
# Assumption: All features are independent of each other given the class (Lung).
# Note: For classification, features are parents of the class node, and features are marginally independent.
# We will model the joint distribution P(L, F) = P(L) * P(F1|L) * P(F2|L) * ...
NAIVE_BAYES_EDGES = {
    'lung': [], # Lung is the root node
    # All other nodes (features) are children of 'lung'
    'asia': ['lung'], 
    'tub': ['lung'], 
    'smoke': ['lung'], 
    'bronc': ['lung'], 
    'either': ['lung'], 
    'xray': ['lung'], 
    'dysp': ['lung'] 
}

# --- 6. Classification Prediction Logic ---

def predict_classification(bn, data, target_node='lung'):
    """
    Predicts the target variable using P(Target | Features) ∝ P(Target, Features)
    P(Target, Features) is calculated by multiplying CPTs.
    """
    
    # Identify feature nodes (all nodes except the target)
    feature_nodes = [n for n in bn.nodes if n != target_node]
    
    predictions = []
    
    for _, sample in data.iterrows():
        # Calculate P(Target=0 | Features) and P(Target=1 | Features)
        log_prob_0 = 0.0 # log(P(Target=0, Features))
        log_prob_1 = 0.0 # log(P(Target=1, Features))
        
        target_states = [0, 1]
        
        for target_state in target_states:
            current_log_prob = 0.0
            
            # Iterate through all nodes (Target and Features)
            for node in bn.nodes:
                cpts = bn.cpts[node]
                parents = bn.structure[node]
                
                # Determine the value of the node and its parents for this calculation
                
                # If the node is the target, use the current target_state
                node_value = target_state if node == target_node else sample[node]
                
                # If a parent is the target, use the current target_state
                parent_config = tuple(target_state if p == target_node else sample[p] for p in parents)
                
                # Get the probability P(Node | Parents)
                prob = 0.0
                if parents:
                    # Probabilities are stored in a Pandas Series or dict indexed by parent_config
                    prob_series = cpts['P_1'] if node_value == 1 else cpts['P_0']
                    
                    if isinstance(prob_series, pd.Series):
                        # Ensure we handle unseen parent configurations by adding a small value
                        try:
                            prob = prob_series.loc[parent_config]
                        except KeyError:
                            # If a parent config was not seen in training, use a small default (Laplace=1 should prevent this)
                            prob = 1e-6 
                    else: # If cpts are simple dictionaries (priors only, not series)
                         prob = prob_series
                else:
                    # Node has no parents (prior probability)
                    prob = cpts['P_1'] if node_value == 1 else cpts['P_0']
                
                current_log_prob += math.log(prob)
            
            if target_state == 0:
                log_prob_0 = current_log_prob
            else:
                log_prob_1 = current_log_prob
                
        # Decision: P(T=1|F) > P(T=0|F)
        # Since log(P(T=1, F)) vs log(P(T=0, F)) is equivalent, we compare log_prob
        prediction = 1 if log_prob_1 > log_prob_0 else 0
        predictions.append(prediction)
        
    return np.array(predictions)

# --- 7. Execution of Classification Experiment (Part b) ---

# 1. Select Optimal K2 Structure (Order 3, Max P=2)
# We need to re-run K2 algorithm to get the actual model object, since we only saved the score.
# This assumes the k2_algorithm defined previously is available.
# Since the actual object was not saved, we redefine the optimal structure found in the logs:
# Order 3: ['smoke', 'asia', 'lung', 'tub', 'bronc', 'either', 'dysp', 'xray']
# Max P: 2
ORDER_3 = ['smoke', 'asia', 'lung', 'tub', 'bronc', 'either', 'dysp', 'xray']
best_bn_k2 = k2_algorithm(df_train, ORDER_3, max_parents=2) # Retrain on df_train

# 2. Define the other two models (BN_True and BN_Naive)
bn_true = BayesianNetwork(COLUMNS)
bn_true.structure = TRUE_STRUCTURE_EDGES

# For Naive Bayes, we must define the node ordering for the network
bn_naive_nodes = ['lung', 'asia', 'tub', 'smoke', 'bronc', 'either', 'xray', 'dysp']
bn_naive = BayesianNetwork(bn_naive_nodes)
bn_naive.structure = {n: NAIVE_BAYES_EDGES.get(n, []) for n in bn_naive_nodes}
# The Naive Bayes structure as defined in the dictionary (Lung is independent, features depend on Lung)
# We must ensure the actual nodes in the BN match the columns and the structure dictionary

# Fix Naive Bayes structure:
bn_naive = BayesianNetwork(COLUMNS)
# Naive Bayes assumes features are independent *given the class*.
# We enforce this by defining features as children of the target ('lung'), 
# and setting their structures based on the target.
naive_structure = {'lung': []}
for node in COLUMNS:
    if node != 'lung':
        # Feature node depends *only* on the target node
        naive_structure[node] = ['lung'] 
bn_naive.structure = naive_structure
bn_naive.nodes = COLUMNS


# 3. Combine Train and Validation data for training (as requested)
df_train_val = pd.concat([df_train, df_val]).reset_index(drop=True)

# 4. Train CPTs on df_train_val
best_bn_k2.learn_cpts(df_train_val)
bn_true.learn_cpts(df_train_val)
bn_naive.learn_cpts(df_train_val)

# 5. Evaluate on Test data (df_test)
y_true = df_test['lung'].values

models = {
    "K2 Optimal (Max P=2)": best_bn_k2,
    "True Structure (ASIA)": bn_true,
    "Naive Bayes (Lung Target)": bn_naive
}

classification_results = {}
print("\n--- Running Classification on Test Data (Target: Lung) ---")

for name, model in models.items():
    y_pred = predict_classification(model, df_test, target_node='lung')
    metrics = calculate_metrics(y_true, y_pred) # Reusing the F1 function from Q3/part 1
    
    # Add Accuracy, Precision, Recall calculation to calculate_metrics if needed, 
    # but for now, we extend the calculation here:
    
    # Recalculate full metrics here for completeness (since calculate_metrics was F1-only before)
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    
    accuracy = (tp + tn) / len(y_true) if len(y_true) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    classification_results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score
    }
    
    print(f"{name}: Acc={accuracy:.4f}, F1={f1_score:.4f}")

# 6. Display Results Table

df_class_results = pd.DataFrame(classification_results).T
print("\n--- Classification Results Summary (Test Data) ---")
print(df_class_results.to_markdown()) 

# The image tag for the plot goes here
#


--- Running Classification on Test Data (Target: Lung) ---
K2 Optimal (Max P=2): Acc=0.9933, F1=0.8837
True Structure (ASIA): Acc=0.9933, F1=0.8837
Naive Bayes (Lung Target): Acc=0.9693, F1=0.6102

--- Classification Results Summary (Test Data) ---


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.